In [1]:
import pandas as pd
from pyarrow import feather
import os
import requests
from bs4 import BeautifulSoup
import re
import time
import concurrent.futures
from github import Github

In [2]:
# Takes a directory and finds all github links referenced within its files
def find_gits(dirpath):
    gitlinks = []
    for file in os.listdir(dirpath):
        f = os.path.join(dirpath, file)
        if os.path.isfile(f) and 'json' in f:
            with open(f) as file:
                lines = file.readlines()
                for line in lines:
                    if "github" in line and "url\" :" in line:
                        line = line.replace("\"url\" : \"", "").replace("\",", "").strip()
                        gitlinks.append(line)
    return gitlinks

In [3]:
# Find all git links in data directory files
gitlinks = find_gits('data')

In [4]:
len(gitlinks)

40471

In [5]:
# Find referenced CVEs and Languages used in Git Repos
def get_data(link):
    cve_exp = 'CVE-([A-Za-z0-9\-\_]+)'
    repo_exp = '.*github.com(\/).*?(\/).*?(\/)'
    loc_exp = '(?<=github.com)(\/).*?(\/).*?(\/)'
    cve1 = 'null'
    repo = 'null'
    cve2 = 'null'
    langsl = 'null'
    contributors = 'null'
    
    try:
        page = requests.get(link)
        text = page.text
        found = re.findall(cve_exp, text)
        cve1 = list(set(found))
        data = link, cve1, repo, cve2, langsl, contributors
    except:
        data = link, cve1, repo, cve2, langsl, contributors

    try:
        slash_count = link.count('/')
        if slash_count <= 4:
            repo = f"{link}/"
            repo_page = requests.get(repo)
            repo_text = repo_page.text
            found = re.findall(cve_exp, repo_text)
            cve2 = list(set(found))
            data = link, cve1, repo, cve2, langsl, contributors
        else:
            repo = re.search(repo_exp, link).group(0)
            repo_page = requests.get(repo)
            repo_text = repo_page.text
            found = re.findall(cve_exp, repo_text)
            cve2 = list(set(found))
            data = link, cve1, repo, cve2, langsl, contributors
    except:
        data = link, cve1, repo, cve2, langsl, contributors

    try:
        soup = BeautifulSoup(repo_page.content, 'html.parser')
        langs = soup.find_all("span", {"class":"Progress-item color-bg-success-emphasis"})
        langsl = []
        for l in langs:
            langsl.append(l['aria-label'])
        data = link, cve1, repo, cve2, langsl, contributors
    except:
        data = link, cve1, repo, cve2, langsl, contributors

    try:
        loc = re.search(loc_exp, repo).group(0)
        url = f"https://api.github.com/repos{loc}contributors"
        c_json = requests.get(url).json()
        contributors = []
        for c in c_json:
            contributors.append(c["login"])
        data = link, cve1, repo, cve2, langsl, contributors
    except:
        data = link, cve1, repo, cve2, langsl, contributors
    
    return data

In [6]:
x = get_data('https://github.com/jbaines-r7/staystaystay')
x

('https://github.com/jbaines-r7/staystaystay',
 ['2021-1585'],
 'https://github.com/jbaines-r7/staystaystay/',
 ['2021-1585'],
 ['C++ 94.6', 'Java 4.0', 'Other 1.4'],
 [])

In [7]:
data = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    data = []
    for link in gitlinks:
        futures.append(executor.submit(get_data, link=link))
    for future in concurrent.futures.as_completed(futures):
        # print(future.result())
        data.append(future.result())

/opt/anaconda3/envs/etlfinal/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [8]:
dd = []
for i in data:
    if i is not None:
        ii = list(i)
        dd.append(ii)
df = pd.DataFrame(dd, columns = ['Link', 'CVEs', 'Repo', 'Repo CVEs', 'Repo Languages', 'Contributors'])

In [9]:
df.tail(10)

,Link,CVEs,Repo,Repo CVEs,Repo Languages,Contributors
40461,https://github.com/grafana/synthetic-monitorin...,[],https://github.com/grafana/synthetic-monitorin...,[],[],[]
40462,https://github.com/discourse/discourse-bbcode/...,[],https://github.com/discourse/discourse-bbcode/,[],[],[]
40463,https://github.com/discourse/discourse/securit...,[],https://github.com/discourse/discourse/,[],[],[]
40464,https://github.com/discourse/discourse-bbcode/...,[],https://github.com/discourse/discourse-bbcode/,[],[],[]
40465,https://github.com/discourse/discourse/commit/...,[],https://github.com/discourse/discourse/,[],[],[]
40466,https://github.com/clastix/capsule/commit/1df4...,[],https://github.com/clastix/capsule/,[],[],[]
40467,https://github.com/clastix/capsule/commit/7552...,[],https://github.com/clastix/capsule/,[],[],[]
40468,https://github.com/clastix/capsule/security/ad...,[],https://github.com/clastix/capsule/,[],[],[]
40469,https://github.com/clastix/capsule/releases/ta...,[],https://github.com/clastix/capsule/,[],[],[]
40470,https://github.com/MatMoul/g810-led/pull/297,[],https://github.com/MatMoul/g810-led/,[],[],[]


In [10]:
uni = df.loc[df.astype(str).drop_duplicates().index]
uni

,Link,CVEs,Repo,Repo CVEs,Repo Languages,Contributors
0,https://github.com/jbaines-r7/staystaystay,[2021-1585],https://github.com/jbaines-r7/staystaystay/,[2021-1585],"[C++ 94.6, Java 4.0, Other 1.4]",[]
1,https://github.com/ansible-collections/communi...,[],https://github.com/ansible-collections/communi...,[],"[Python 99.4, Other 0.6]",[]
2,https://github.com/orangecertcc/security-resea...,[2021-0253],https://github.com/orangecertcc/security-resea...,"[2019-17570, 2021-42371, 2021-42372, YYYY-XXXX...","[Python 80.8, Java 18.4, Makefile 0.8]",[]
3,https://github.com/BlackFan/client-side-protot...,[2021-20089],https://github.com/BlackFan/client-side-protot...,"[2021-20088, 2021-20084, 2021-20085, 2021-2008...",[],[]
4,https://github.com/BlackFan/client-side-protot...,[2021-20087],https://github.com/BlackFan/client-side-protot...,"[2021-20088, 2021-20084, 2021-20085, 2021-2008...",[],[]
...,...,...,...,...,...,...
40466,https://github.com/clastix/capsule/commit/1df4...,[],https://github.com/clastix/capsule/,[],[],[]
40467,https://github.com/clastix/capsule/commit/7552...,[],https://github.com/clastix/capsule/,[],[],[]
40468,https://github.com/clastix/capsule/security/ad...,[],https://github.com/clastix/capsule/,[],[],[]
40469,https://github.com/clastix/capsule/releases/ta...,[],https://github.com/clastix/capsule/,[],[],[]


In [11]:
uni.to_pickle('gitdata.pickle')